<a href="https://colab.research.google.com/github/twochae2026/Hecto-AI-Challenge/blob/main/Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ---------- 0. install ----------
!pip install -q timm opencv-python pandas

# ---------- 1. import ----------
import os
import cv2
import glob
import torch
import timm
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# ---------- 2. device ----------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

# ---------- 3. model ----------
class DeepfakeDetector(nn.Module):
    def __init__(self, backbone="convnext_base"):
        super().__init__()
        self.backbone = timm.create_model(
            backbone,
            pretrained=False,
            num_classes=0
        )
        self.head = nn.Linear(self.backbone.num_features, 1)

    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)
        return x.squeeze(1)

# ---------- 4. preprocess ----------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# ---------- 5. dataset ----------
class ImageDataset(Dataset):
    def __init__(self, paths):
        self.paths = paths

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        return transform(img)

# ---------- 6. load model ----------
# model.pth를 Colab Files에 업로드해야 함
# MODEL_PATH = "/content/model.pth"

# model = DeepfakeDetector("convnext_base")
# model.load_state_dict(torch.load(MODEL_PATH, map_location="cpu"))
# model.to(DEVICE)
# model.eval()

print("Model loaded")

model = DeepfakeDetector("convnext_base")
model.to(DEVICE)
model.eval()

print("⚠️ model.pth 없음 → 랜덤 가중치로 실행")


# ---------- 7. image inference ----------
def infer_images(image_paths, batch_size=32):
    dataset = ImageDataset(image_paths)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    scores = []

    with torch.no_grad():
        for imgs in loader:
            imgs = imgs.to(DEVICE)
            logits = model(imgs)
            probs = torch.sigmoid(logits)
            scores.extend(probs.cpu().numpy())

    return scores

# ---------- 8. video inference ----------
def infer_video(
    video_path,
    frame_stride=5,
    top_k_ratio=0.2
):
    cap = cv2.VideoCapture(video_path)
    frames = []
    idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if idx % frame_stride == 0:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
        idx += 1

    cap.release()

    if len(frames) == 0:
        return 0.0

    imgs = torch.stack([
        transform(Image.fromarray(f)) for f in frames
    ]).to(DEVICE)

    with torch.no_grad():
        probs = torch.sigmoid(model(imgs)).cpu().numpy()

    k = max(1, int(len(probs) * top_k_ratio))
    return float(np.mean(np.sort(probs)[-k:]))

# ---------- 9. run inference ----------
results = {}

# 경로는 대회 제공 구조에 맞게 조정
IMAGE_DIR = "/content/test_image"
VIDEO_DIR = "/content/test_video"

image_files = glob.glob(os.path.join(IMAGE_DIR, "*"))
video_files = glob.glob(os.path.join(VIDEO_DIR, "*"))

print(f"Images: {len(image_files)}, Videos: {len(video_files)}")

# images
if len(image_files) > 0:
    image_scores = infer_images(image_files)
    for path, score in zip(image_files, image_scores):
        results[os.path.basename(path)] = score

# videos
for v in video_files:
    results[os.path.basename(v)] = infer_video(v)

# ---------- 10. save submission ----------
df = pd.DataFrame(
    results.items(),
    columns=["id", "score"]
)

df.to_csv("submission.csv", index=False)
print("submission.csv saved")
df.head()


DEVICE: cuda
Model loaded
⚠️ model.pth 없음 → 랜덤 가중치로 실행
Images: 245, Videos: 500
submission.csv saved


,id,score
0,TEST_107.jpeg,0.632763
1,TEST_091.png,0.441483
2,TEST_206.jpg,0.604663
3,TEST_269.jpg,0.549894
4,TEST_309.jpg,0.424467


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['score'].plot(kind='hist', bins=20, title='score')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('id').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_7['score'].plot(kind='line', figsize=(8, 4), title='score')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_8['id'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_8, x='index', y='id', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_9['id'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_9, x='score', y='id', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
import pandas as pd

df = pd.read_csv("/content/sample.csv")
df


,filename,prob
0,TEST_000.mp4,0
1,TEST_001.jpg,0
2,TEST_002.mp4,0
3,TEST_003.mp4,0
4,TEST_004.jpg,0
...,...,...
495,TEST_495.jpg,0
496,TEST_496.jpg,0
497,TEST_497.mp4,0
498,TEST_498.mp4,0
